In [ ]:
import packages.Preprocesamiento as ppr
import os

In [ ]:
# Load
path_datos = os.path.join('Datos','Originales')
filename = os.path.join(path_datos,'cancellation_data.csv')
df = ppr.read_data(filename)
#print(df.head())